# Predicting Diabetes Prognosis

## Introduction:

Diabetes is a medical condition where the afflicted body is unable to create enough insulin to efficiently transform glucose into energy for the body's utilization. If left undiagnosed and untreated, diabetes can inflict serious harm to a person's heart and blood vessels, eyes, kidneys, nerves, gastrointestinal tract, and gums and teeth. Given the health risks of untreated diabetes, it is imperative that medical professionals have a method of diagnosing the condition as soon as possible. This lead us to our question: *Is it possible to diagnose diabetes in patients based on their medical factors? If so, what medical factors should we use to create the most accuracte predictive model?* To answer these questions, this project intends to create a k-nn classifier model that can predict whether or not a person is diabetic based on their medical factors. The data set that we will use to create this classifier is the Pima Indians Diabetes Dataset, which we obtained from https://github.com/npradaschnor/Pima-Indians-Diabetes-Dataset/blob/master/diabetes.csv. This dataset contains the health information of 768 women from Phoenix, Arizona. 

The dataset contains 9 columns:
- Outcome (1 is diabetic, 0 is not diabetic)
- Number of pregnancies
- Glucose tolerance (two hour plasma glucose concentration after 75g anhydrous glucose in mg/dl)
- Blood pressure (Diastolic BP in mmHg)
- Skin thickness (Triceps skin fold thickness in mm)
- Insulin (2 hour serum insulin in mu U/ml)
- BMI (Body Mass Index)
- Pedigree diabetes function
- Age (years)

For our classifier, Diabetes diagnosis will be the target variable and we will choose the predictor variables using forward selection to ensure that we are using the combination of predictor variables that yields the best prediction accuracy.

## Methods and Results

### Our Process

Below is an overview of our data analysis process (a more detailed explaination will be provided at each step)
1) Read and Wrangle the dataset into a tidy dataset.
2) Split the tidy dataset into a training and testing set. 
3) Use forward selection to find the combination of predictor variables with the highest prediction accuracy. 
4) Perform k-nearest neighbor classification with the best value of k found using cross-validation and predict the test set to get the final model accuracy validation.
5) Describe our results and conclusion.

### Loading Relevant Libraries

In [3]:
#RUN THIS CELL 
library(tidyverse)
library(tidymodels)
library(forcats)
library(RColorBrewer)
library(repr)
library(ggplot2)
library(knitr)
options(repr.matrix.max.rows = 6)

## Read and wrangle the dataset from the web into jupyter as a tidy dataset

* We bring the data set from the web into Jupyter using <code>read_csv()</code>. 
* The Outcome column will be our target variable so we make it into a factor variable using <code>as_factor()</code>.
* The resulting dataset is our wrangled <code>diabetes_data</code> with 768 usable observations.

In [8]:
data <- read_csv("https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv")

diabetes_data <- data |>
mutate(Outcome = as_factor(Outcome))

diabetes_data

Rows: 768 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, D...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
5,121,72,23,112,26.2,0.245,30,0
1,126,60,0,0,30.1,0.349,47,1
1,93,70,31,0,30.4,0.315,23,0


### Summarizing the Training Data

* As a result of our wrangling data step, we know that our data set contains a total of 768 usuable observations
* We have decided to perform a 